In [2]:
from tensorflow.keras.datasets import cifar10,cifar100
import tensorflow.keras.layers as tfl
from tensorflow.keras import Model, Input, utils
from matplotlib.pyplot import imshow, show
import numpy as np
from cv2 import imread, imwrite, resize
import matplotlib.pyplot as plt
from os.path import join
from os import listdir
from tensorflow.keras.applications import MobileNetV3Small, MobileNetV2, MobileNet, mobilenet_v3
from tensorflow import concat, reshape, GradientTape
from tensorflow.keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from tensorflow import one_hot
from tensorflow.data import Dataset


In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [254]:
model_backbone = MobileNetV3Small(
    input_shape=(224,224,3),
    alpha=1.0,
    minimalistic=False,
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    classes=1000,
    pooling=None,
    dropout_rate=0.2,
    classifier_activation='softmax',
    include_preprocessing=True
)

In [3]:
# model_backbone.summary()

In [6]:
class HierarchicalModel(Model):
    def __init__(self, inputs, outputs):
        super().__init__(inputs, outputs)

    # def compile(self, optimizer, lamb, sigma, metrics=None,**kwargs):
    #     super(HierarchicalModel, self).compile(optimizer=optimizer, metrics=metrics, **kwargs)
    #     self.lamb = lamb
    #     self.sigma = sigma
    def set_hyper(self, lamb, sigma):
        self.lamb = lamb
        self.sigma = sigma
    def train_step(self, data):
        x, y = data
        y_true_category = y['category']
        y_true_grade = y['grade']
        with GradientTape() as tape:
            [y_pred_category, y_pred_grade] = self(x, training=True)
            category_loss = categorical_crossentropy(y_true_category, y_pred_category)
            grade_loss = categorical_crossentropy(y_true_grade, y_pred_grade)
            combined_loss = (1 - self.lamb) * category_loss + self.lamb * grade_loss + self.sigma * category_loss * grade_loss

        # Compute gradients
        gradients = tape.gradient(combined_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, [y_pred_category, y_pred_grade])

        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [8]:
def backbone_model(
    image_shape = (224, 224 , 3),
    category_depth = 9,
    trainable = False):

    backbone = MobileNetV3Small(
    input_shape= image_shape,
    include_top= False,
    weights= 'imagenet',
    input_tensor= None,
    dropout_rate= 0.2,
    include_preprocessing= True)

    backbone.trainable = trainable

    last_layer = backbone.get_layer(backbone.layers[-1].name).output

    conv_output = backbone.get_layer(f'expanded_conv_{str(category_depth)}/Add').output

    return Model(inputs = backbone.input, outputs = [last_layer, conv_output])


In [10]:
def hierarchical_model(
    image_shape = (224, 224 , 3),
    category_depth = 9,
    category_num = 2,
    grade_num = 2):


    backbone = backbone_model(
    image_shape = image_shape,
    category_depth = category_depth,
    trainable = False)

    backbone.trainable = False

    input = Input(shape = image_shape)

    [x, conv_output] = backbone(input, training = False)

    flatten = tfl.Flatten()(x)
    x_category = tfl.Dense(16, activation = 'relu')(flatten)
    category_output = tfl.Dense(category_num, activation = 'softmax', name = 'category')(x_category)


    # x_grade = tfl.Flatten()(x)
    x_grade = tfl.Dense((16-category_num), activation = 'relu')(flatten)
    x_grade = concat([x_grade, category_output], -1)
    grade_output = tfl.Dense(grade_num, activation = 'softmax', name = 'grade')(x_grade)

    model = HierarchicalModel(inputs = input, outputs = [category_output, grade_output])

    return model






In [264]:
model = hierarchical_model()
model.summary()

Model: "hierarchical_model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_67 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_43 (Functional)          [(None, 7, 7, 576),  939120      ['input_67[0][0]']               
                                 (None, 7, 7, 96)]                                                
                                                                                                  
 flatten_53 (Flatten)           (None, 28224)        0           ['model_43[0][0]']               
                                                                              

In [12]:
# import tensorflow as tf
# @tf.function
def combined_loss(y_true, y_pred, lamb=0.5):
    print(y_true.shape)
    print(y_pred.shape)
    y_true_category, y_true_grade = y_true[0, :], y_true[1, :]
    y_pred_category, y_pred_grade = y_pred[0, :], y_pred[1, :]

    category_loss = categorical_crossentropy(y_true_category, y_pred_category)
    grade_loss = categorical_crossentropy(y_true_grade, y_pred_grade)

    return (1 - lamb) * category_loss + lamb * grade_loss + 5* category_loss * grade_loss

In [18]:
utils.plot_model(model, show_shapes=True, show_dtype=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [176]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

file_path = 'labeled.h5'

with h5py.File(file_path, 'r') as hf:
    x_train = hf['images'][()]
    y_category = hf['labels'][()]
    y_grade = hf['qualities'][()]
print(x_train.shape)
print(y_category.shape)
print(y_grade.shape)
print(set(y_category))

(2000, 224, 224, 3)
(2000,)
(2000,)
{0, 1}


In [33]:

y_category = reshape(one_hot(y_category,2),(-1,2))
y_grade = reshape(one_hot(y_grade,2),(-1,2))
print(y_category.shape)
print(y_grade.shape)

(2000, 2)
(2000, 2)


In [42]:
dataset = Dataset.from_tensor_slices((x_train, {'category': y_category, 'grade': y_grade}))
dataset = dataset.shuffle(buffer_size=2000, reshuffle_each_iteration=True)
dataset = dataset.batch(32)

In [148]:
model.compile(optimizer = Adam(learning_rate=1e-5),loss='categorical_crossentropy', metrics = ['accuracy'])#, lamb = 0.5, sigma = 5)
model.set_hyper(lamb = 0.5, sigma = 0)
model.fit(dataset, epochs = 10)

Epoch 1/10
63/63 [==============================] - 15s 198ms/step - category_accuracy: 0.5525 - grade_accuracy: 1.0000
Epoch 2/10
63/63 [==============================] - 12s 193ms/step - category_accuracy: 0.6670 - grade_accuracy: 1.0000
Epoch 3/10
63/63 [==============================] - 12s 194ms/step - category_accuracy: 0.7640 - grade_accuracy: 1.0000
Epoch 4/10
63/63 [==============================] - 12s 193ms/step - category_accuracy: 0.8240 - grade_accuracy: 1.0000
Epoch 5/10
63/63 [==============================] - 12s 194ms/step - category_accuracy: 0.8740 - grade_accuracy: 1.0000
Epoch 6/10
63/63 [==============================] - 12s 195ms/step - category_accuracy: 0.9110 - grade_accuracy: 1.0000
Epoch 7/10
63/63 [==============================] - 12s 195ms/step - category_accuracy: 0.9350 - grade_accuracy: 1.0000
Epoch 8/10
63/63 [==============================] - 12s 195ms/step - category_accuracy: 0.9525 - grade_accuracy: 1.0000
Epoch 9/10
63/63 [======================

In [21]:
def flat_model(
    image_shape = (224, 224 , 3),
    num_class = 4
):
    backbone = backbone_model(
    image_shape = image_shape,
    trainable = False)

    backbone.trainable = False

    input = Input(shape = image_shape)

    [last_layer, conv_output] = backbone(input, training = False)

    X = tfl.Flatten()(last_layer)
    X = tfl.Dense(32, activation = 'relu')(X)
    X = tfl.Dense(16, activation = 'relu')(X)
    outputs = tfl.Dense(num_class, activation = 'softmax', name = 'category')(X)

    model = Model(inputs = input, outputs = outputs)

    return model


In [23]:
cardinary_model = flat_model(num_class = 12)
cardinary_model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 model_4 (Functional)        [(None, 7, 7, 576),       939120    
                              (None, 7, 7, 96)]                  
                                                                 
 flatten_2 (Flatten)         (None, 28224)             0         
                                                                 
 dense_3 (Dense)             (None, 32)                903200    
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 category (Dense)            (None, 12)                204       
                                                           

In [188]:
file_path = 'new_labeled.h5'

with h5py.File(file_path, 'r') as hf:
    X = hf['images'][()]
    Labels = hf['units'][()]
print(X.shape)
print(Labels.shape)
Labels = reshape(one_hot(Labels,4),(-1,4))
print(Labels.shape)
for i in Labels.numpy():
    print(i)
print(Labels)

(2000, 224, 224, 3)
(2000,)
(2000, 4)
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 1.]
[0. 0. 0. 

In [160]:
train_set = Dataset.from_tensor_slices((X, Labels))
train_set = train_set.shuffle(buffer_size=2000, reshuffle_each_iteration=True)
train_set = train_set.batch(32)

In [162]:
cardinary_model.compile(optimizer = Adam(learning_rate=1e-5), loss = 'categorical_crossentropy', metrics = ['accuracy'])
cardinary_model.fit(train_set, epochs = 15)

Epoch 1/15
63/63 [==============================] - 15s 191ms/step - loss: 0.9398 - accuracy: 0.3590
Epoch 2/15
63/63 [==============================] - 12s 188ms/step - loss: 0.5283 - accuracy: 0.6220
Epoch 3/15
63/63 [==============================] - 12s 187ms/step - loss: 0.3011 - accuracy: 0.7110
Epoch 4/15
63/63 [==============================] - 12s 188ms/step - loss: 0.1838 - accuracy: 0.7480
Epoch 5/15
63/63 [==============================] - 12s 187ms/step - loss: 0.1273 - accuracy: 0.7670
Epoch 6/15
63/63 [==============================] - 12s 187ms/step - loss: 0.0958 - accuracy: 0.7830
Epoch 7/15
63/63 [==============================] - 12s 188ms/step - loss: 0.0778 - accuracy: 0.7795
Epoch 8/15
63/63 [==============================] - 12s 190ms/step - loss: 0.0677 - accuracy: 0.7720
Epoch 9/15
63/63 [==============================] - 17s 244ms/step - loss: 0.0566 - accuracy: 0.7755
Epoch 10/15
63/63 [==============================] - 15s 245ms/step - loss: 0.0503 - accura

In [134]:
cardinary_model.fit(X, Labels, epochs = 15)

Epoch 1/15
63/63 [==============================] - 18s 242ms/step - loss: 0.0046 - accuracy: 0.8575
Epoch 2/15
63/63 [==============================] - 15s 241ms/step - loss: 0.0046 - accuracy: 0.8555
Epoch 3/15
63/63 [==============================] - 15s 244ms/step - loss: 0.0046 - accuracy: 0.8520
Epoch 4/15
63/63 [==============================] - 15s 242ms/step - loss: 0.0046 - accuracy: 0.8595
Epoch 5/15
63/63 [==============================] - 15s 238ms/step - loss: 0.0046 - accuracy: 0.8565
Epoch 6/15
63/63 [==============================] - 15s 239ms/step - loss: 0.0046 - accuracy: 0.8550
Epoch 7/15
63/63 [==============================] - 15s 245ms/step - loss: 0.0045 - accuracy: 0.8575
Epoch 8/15
63/63 [==============================] - 15s 241ms/step - loss: 0.0046 - accuracy: 0.8575
Epoch 9/15
63/63 [==============================] - 15s 241ms/step - loss: 0.0045 - accuracy: 0.8565
Epoch 10/15
63/63 [==============================] - 15s 242ms/step - loss: 0.0046 - accura

In [32]:
outputs = model.predict(x_train)


63/63 [==============================] - 9s 133ms/step


In [34]:
[pred_category, pred_grade] = outputs

In [166]:
tf.config.list_physical_devices('GPU')

[]

In [36]:
print(pred_category.shape)

(2000, 2)


In [38]:
print(y_true)

NameError: name 'y_true' is not defined

In [102]:
def accuracy(y_true, y_pred, threshold = 0.5):
    y_pred = np.where(y_pred > threshold, 1, 0)
    return np.sum(y_true == y_pred) / len(y_true)

In [48]:
import tensorflow as tf
@tf.function
def custom_accuracy(y_true, y_pred):
    # Unpack true labels
    y_true_category = y_true[0,:]
    y_true_grade = y_true[1,:]

    # Unpack predicted labels
    y_pred_category = y_pred[0,:]
    y_pred_grade = y_pred[1,:]

    # Get the predicted class (highest probability) for both outputs
    y_pred_category = tf.argmax(y_pred_category, axis=-1)
    y_pred_grade = tf.argmax(y_pred_grade, axis=-1)

    # Get the true class for both outputs
    y_true_category = tf.argmax(y_true_category, axis=-1)
    y_true_grade = tf.argmax(y_true_grade, axis=-1)

    # Compare both predictions with true labels
    is_correct_category = tf.equal(y_pred_category, y_true_category)
    is_correct_grade = tf.equal(y_pred_grade, y_true_grade)

    # Only count as correct if both category and grade are correct
    is_correct = tf.logical_and(is_correct_category, is_correct_grade)
    # for i in range(len(is_correct)):
    #   print(is_correct[i].numpy(),'Category:', is_correct_category[i].numpy(),"Grade", is_correct_grade[i].numpy())
    # Convert boolean tensor to float and take the mean to get accuracy
    return tf.reduce_mean(tf.cast(is_correct, tf.float32))

def category_accuracy(y_true, y_pred):
    y_true_category = tf.argmax(y_true, axis=-1)
    y_pred_category = tf.argmax(y_pred, axis=-1)
    is_true = tf.equal(y_true_category, y_pred_category)
    print(is_true)
    return tf.reduce_mean(tf.cast(is_true, tf.float32))

def grade_accuracy(y_true, y_pred):
    y_true_grade = tf.argmax(y_true, axis=-1)
    y_pred_grade = tf.argmax(y_pred, axis=-1)
    is_true = tf.equal(y_true_grade, y_pred_grade)
    print(is_true)
    return tf.reduce_mean(tf.cast(is_true, tf.float32))

In [52]:
def custom_accuracy(y_true, y_pred):
    # Unpack true labels
    y_true_category = y_true[0]
    y_true_grade = y_true[1]

    # Unpack predicted labels
    y_pred_category = y_pred[0]
    y_pred_grade = y_pred[1]

    # Get the predicted class (highest probability) for both outputs
    y_pred_category = tf.argmax(y_pred_category, axis=-1)
    y_pred_grade = tf.argmax(y_pred_grade, axis=-1)

    # Get the true class for both outputs
    y_true_category = tf.argmax(y_true_category, axis=-1)
    y_true_grade = tf.argmax(y_true_grade, axis=-1)

    # Compare both predictions with true labels
    is_correct_category = tf.equal(y_pred_category, y_true_category)
    is_correct_grade = tf.equal(y_pred_grade, y_true_grade)

    # Only count as correct if both category and grade are correct
    is_correct = tf.logical_and(is_correct_category, is_correct_grade)
    for i in range(len(is_correct)):
        print(is_correct[i].numpy(),'Category:', is_correct_category[i].numpy(),"Grade", is_correct_grade[i].numpy())
    # Convert boolean tensor to float and take the mean to get accuracy
    return tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [54]:
print(custom_accuracy([y_category,y_grade], [pred_category, pred_grade]))
print(category_accuracy(y_category, pred_category))
print(grade_accuracy(y_grade, pred_grade))

True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Category: True Grade True
True Cat

In [ ]:
print(y_category[0], y_grade[0])
# imshow(x_train[0])


tf.Tensor([1. 0.], shape=(2,), dtype=float32) tf.Tensor([1. 0.], shape=(2,), dtype=float32)
